# day 15 part two

Finding the unique node that could contain a beacon

In [2]:
import numpy as np
import re

def load_input(fp):
    with open(fp) as f_in:
        for line in f_in.read().splitlines():
            yield line

def make_cmplx(line: str, pattern: re.compile) -> tuple:
    """Parse the input line containing two pairs of (x, y) coords
    into pair of complex numbers representing the sensor and beacon coordinates
    """
    sx, sy, bx, by = list(map(int, pattern.findall(line)))
    return complex(sx, sy), complex(bx, by)

def calc_man_dist(s: complex, b: complex) -> int:
    """Returns manhattan distance given two complex numbers
    that represent their cartesian coordinates
    """
    return int(abs((b - s).real) + abs((b - s).imag))

In [3]:
test_in = "test.txt"
run_in = "input.txt"

In [84]:
pattern = re.compile("-?\d+")
test_pairs = np.array([make_cmplx(line, pattern) for line in load_input(test_in)])
sb_pairs = np.array([make_cmplx(line, pattern) for line in load_input(run_in)])

In [19]:
test_pairs

array([[ 2.+18.j, -2.+15.j],
       [ 9.+16.j, 10.+16.j],
       [13. +2.j, 15. +3.j],
       [12.+14.j, 10.+16.j],
       [10.+20.j, 10.+16.j],
       [14.+17.j, 10.+16.j],
       [ 8. +7.j,  2.+10.j],
       [ 2. +0.j,  2.+10.j],
       [ 0.+11.j,  2.+10.j],
       [20.+14.j, 25.+17.j],
       [17.+20.j, 21.+22.j],
       [16. +7.j, 15. +3.j],
       [14. +3.j, 15. +3.j],
       [20. +1.j, 15. +3.j]])

In [85]:
dists = np.array([calc_man_dist(*sb) for sb in sb_pairs])
test_dists = np.array([calc_man_dist(*sb) for sb in test_pairs])
test_dists

array([ 7,  1,  3,  4,  4,  5,  9, 10,  3,  8,  6,  5,  1,  7])

## Find all points outside each sensor's detection range

Given sensor and the range, generate the points outside the detection zone

In [76]:
def find_perimeter(sensor: complex, reach: int) -> np.array:
    """
    Given sensor and the range, generate the perimeter points 
    outside the detection zone
    This method really lends itself well to a 45 deg rotation...
    find two at a time;
    1. origin a: left vertex, origin b: bottom vertex
       progress at 45 deg: x++, y--
       stop when pos.x == sensor.x
    2. c: left vertex, d: top
       progress at -45 deg: x++, y++
       stop when pos.x == sensor.x
    """
    
    left = sensor - reach - 1
    bot = sensor + (reach + 1) * 1j
    top = sensor - (reach + 1) * 1j
    right = sensor + reach + 1
    # initialize at vertex
    pos_l = left
    pos_r = right
    pos_b = bot
    pos_t = top
    
    # define dir
    diag_upright = 1 - 1j
    diag_downright = 1 + 1j
    
    while pos_l.real < sensor.real:
        pos_l += 1 - 1j
        pos_t += 1 + 1j
        pos_r += -1 + 1j
        pos_b += -1 - 1j
        
        for pos in [pos_l, pos_t, pos_r, pos_b]:
            yield pos
    # yield left, bot, top, right

In [72]:
foo = 3+4j

In [80]:
gen_nodes = find_perimeter(foo, 1)
# for xy in gen_nodes:
#     print(xy)
print(len(list(gen_nodes)))

8


In [81]:
for i in range(1, 5):
    nodes = list(find_perimeter(foo, i))
    print(f"radius: {i}\t perimeter: {len(nodes)}")

radius: 1	 perimeter: 8
radius: 2	 perimeter: 12
radius: 3	 perimeter: 16
radius: 4	 perimeter: 20


In [86]:
total = 0
for (s, b), d in zip(sb_pairs, dists):
    nodes = list(find_perimeter(s, d))
    num = len(nodes)
    total += num
    print(f"sensor: {s}, search volume: {num}")
print(num)

sensor: (2300471+2016823j), search volume: 4770492
sensor: (1315114+37295j), search volume: 1450248
sensor: (1039523+3061589j), search volume: 4717536
sensor: (214540+3768792j), search volume: 2806532
sensor: (1641345+3524291j), search volume: 1026920
sensor: (1016825+1450262j), search volume: 3283332
sensor: (2768110+3703050j), search volume: 2586580
sensor: (2213658+3522463j), search volume: 3323484
sensor: (3842967+3381135j), search volume: 178428
sensor: (3952516+2683159j), search volume: 3055672
sensor: (172892+369117j), search volume: 5886180
sensor: (3999720+3498306j), search volume: 947740
sensor: (1596187+307084j), search volume: 1355016
sensor: (3863253+3406760j), search volume: 157072
sensor: (3927553+3450758j), search volume: 468880
sensor: (2774120+3228484j), search volume: 1970756
sensor: (3897140+3418751j), search volume: 244656
sensor: (1880329+2843697j), search volume: 3311180
sensor: (33790+3243415j), search volume: 4185040
sensor: (438583+2647769j), search volume: 38

In [87]:
print(total)

91440136


In [88]:
from itertools import product

In [91]:
foo = [1,2,3]; bar = ['a','b','c']
for i in product(foo, bar):
    print(i)

(1, 'a')
(1, 'b')
(1, 'c')
(2, 'a')
(2, 'b')
(2, 'c')
(3, 'a')
(3, 'b')
(3, 'c')
